In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import cross_val_score
import numpy as np
from sklearn import svm
import pandas as pd
import re
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('train.csv', encoding='latin')
df.head()

,Unnamed: 0,author,content,sentiment
0,26898,richardepryor,"@treasaint salad stuff, some chillis, whatever...",happiness
1,27635,reese,"@sunnyjamiel sunny, I'm a workin' on it. It's ...",neutral
2,3036,mutedriposte,@jolynnchew so early??,surprise
3,5604,sakizzie_1102,"So now, I have conjunctivitis in my left eye. ...",sadness
4,36111,poptrash,"Out and about in Deal, Kent. More sunshine req...",love


In [3]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,author,content
0,32823,valicast,Good Morning
1,16298,btb103,I just put my computer up on craigslist. I've ...
2,28505,anavil,in ten minutes shopping demi lovato-back aro...
3,6689,ritwik1st,From twitterberry moved to ubertwitter - suffe...
4,26893,TightFreebies,@thriftymom TEAR*


## Обработка данных <br>
функция processing при надобности убирает <br>
1)ссылки<br>
2)указатели на чьи-то id(например, @jolynnchew)<br>
3)стоп-слова<br>
4)заменяет слова типа "'m", "'ll", "'t", "'re" , "'s" на " am", " will", " not", " are", " is"<br>

находит: <br>
1)кол-во ссылок в сообщении<br>
2)кол-во указаний на имена<br>
3)массив смайлов, присутствующих в предложении <br>
4)кол-во восклицательных знаков <br>
5)кол-во вопросительных знаков <br>
6)кол-во троеточий


In [5]:
#находим кол-во различных смайлов
def smiles(text):
    smiles = []
    smiles_count = np.zeros(15)
    smiles.append(re.findall(r'[A-Za-z\s];\)', text))
    smiles.append(re.findall(r'[A-Za-z\s]:\)', text))
    smiles.append(re.findall(r'[A-Za-z\s]= \)', text))
    smiles.append(re.findall(r'[A-Za-z\s]:\(', text))
    smiles.append(re.findall(r'[A-Za-z\s]:-D', text))
    smiles.append(re.findall(r'[A-Za-z\s]:0', text))
    smiles.append(re.findall(r'[A-Za-z\s]-_-', text))
    smiles.append(re.findall(r'[A-Za-z\s]8\)', text))
    smiles.append(re.findall(r'[A-Za-z\s]:^\)', text))
    smiles.append(re.findall(r'[A-Za-z\s]=\(', text))
    smiles.append(re.findall(r"[A-Za-z\s]=D", text))
    smiles.append(re.findall(r"[A-Za-z\s]\)\)", text))
    smiles.append(re.findall(r"[A-Za-z\s]:3", text))
    smiles.append(re.findall(r"[A-Za-z\s]:\*", text))
    smiles.append(re.findall(r"[A-Za-z\s]:-\*", text))
    
    
    for i, smile in enumerate(smiles):
        smiles_count[i] = len(smile)
        
    return smiles_count


In [6]:
def processing(text, links=True, id_names=True, stop_words=True, apostrophe=True):
    stopwords_ = stopwords.words('english')
    new_text = text
    if links:
        new_text = re.sub(r'https?:?//[A-Za-z0-9/.]+', '', text)
        
    if id_names:
        new_text = re.sub(r'@[A-Za-z0-9]+', '', new_text)
        
    if apostrophe:
        new_text = re.sub("'m", ' am', new_text)
        new_text = re.sub("'ll", ' will', new_text)
        new_text = re.sub("'t", ' not', new_text)
        new_text = re.sub("'re", ' are', new_text)
        new_text = re.sub("'s", ' is', new_text)
        new_text = re.sub("'", '', new_text)
        
    text_words = np.array(re.findall(r"[\w']+", new_text))
    if stop_words:
        text_words = [str(word).lower() for word in text_words if str(word).lower() not in stopwords_]
    
    #считаем кол-во ссылок и указаний на имена 
    count_links = len(re.findall(r'https?:?//[A-Za-z0-9/.]+', text))
    count_id_names = len(re.findall(r'@[A-Za-z0-9]+', text))
    
    #считаем кол-во вопр/воск знаков и троеточий
    count_exclamation_mark = len(re.findall(r'!', text))
    count_question_mark = len(re.findall(r'\?', text))
    count_ellipsis = len(re.findall(r'\.\.\.', text))
    
    #находим массив смайлов
    smiles_count = smiles(text)
    return ' '.join(text_words), count_links, count_id_names, count_exclamation_mark, count_question_mark, count_ellipsis, smiles_count

## Выделяем некоторые фичи
Самыми хорошими на кросс-валидации оказались следующие фичи: <br>
1) длина предложения <br>
2)кол-во указаний на имена <br>
3)кол-во ?<br>
4)кол-во !<br>
5)массив из смайлов<br>

При этом из текстов были удалены указания на имена и апострофная запись заменена на нормальную

In [7]:
from tqdm import tqdm

In [13]:
features_train = []
features_test = []

texts_words = []
texts_words_test = []

for text in tqdm_notebook(df.content):
    text_words, count_links, count_id_names, count_exclamation, count_question, count_ellipsis, smiles_count = processing(text, links=False, id_names=True, stop_words=False, apostrophe=True)
    texts_words.append(text_words)
    features = []
    features.append(len(text_words))
    features.append(count_id_names)
    features.append(count_exclamation)
    features.append(count_question)
    for smile in smiles_count:
        features.append(smile)

    features_train.append(features)

for text in tqdm_notebook(df_test.content):
    text_words, count_links, count_id_names, count_exclamation, count_question, count_ellipsis, smiles_count = processing(text, links=False, id_names=True, stop_words=False, apostrophe=True)
    texts_words_test.append(text_words)
    features = []
    features.append(len(text_words))
    features.append(count_id_names)
    features.append(count_exclamation)
    features.append(count_question)
    for smile in smiles_count:
        features.append(smile)
    features_test.append(features)


## Oбучаем TfidfVectorizer

In [14]:
vect = TfidfVectorizer(min_df=2, max_df=0.85, max_features=3290)
vect.fit(np.hstack((np.array(texts_words), np.array(texts_words_test))))
matrix_train = vect.transform(np.array(texts_words))
matrix_test = vect.transform(np.array(texts_words_test))

features_train = np.hstack((features_train, matrix_train.A))
features_test = np.hstack((features_test, matrix_test.A))

## Кодируем ответы

In [15]:
encoder = LabelEncoder()
y = df.sentiment
y = encoder.fit_transform(y)


Делаем over sampling

In [ ]:
# ratio = {0:300, 1:300, 2:1000, 3:1000, 4:2000, 5:4500, 6:1500, 7:4000, 8:6455, 9:2000, 10:4000, 11:2500, 12:6366}
# sampler = RandomOverSampler(ratio=ratio)
# X, y = sampler.fit_sample(np.array(features_train), y)

## Обучаем логистическую регрессию

In [16]:
clf = LogisticRegression(max_iter=200)
clf.fit(features_train, y)
predictions = clf.predict(features_test)

predictions_sentiment = encoder.inverse_transform(predictions)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Посмотрим на предсказания

In [18]:
predictions_sentiment

array(['happiness', 'worry', 'worry', ..., 'sadness', 'worry', 'neutral'],
      dtype=object)

Запишем ответ в файл

In [19]:
answers = pd.read_csv('sampleSubmission.csv')
answers['sentiment'] = np.array(predictions_sentiment)
answers.to_csv('attempt1.csv',  index=False)
answers.head()

,id,sentiment
0,32823,happiness
1,16298,worry
2,28505,worry
3,6689,neutral
4,26893,neutral
